In [0]:
secret=dbutils.secrets.get('newscope12','scope')

In [0]:
client_id = "88dec17f-62f0-4cab-82b7-b9022c2cc21c"
client_secret =secret
directory_id="037e0179-cd0e-4201-951c-167b7554d77a"

spark.conf.set("fs.azure.account.auth.type.storageforcustomers.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.storageforcustomers.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.storageforcustomers.dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.storageforcustomers.dfs.core.windows.net", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.storageforcustomers.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
display(dbutils.fs.ls("abfss://raw@storageforcustomers.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://raw@storageforcustomers.dfs.core.windows.net/accounts.csv,accounts.csv,2331,1730905206000
abfss://raw@storageforcustomers.dfs.core.windows.net/cleaned_data/,cleaned_data/,0,1730934907000
abfss://raw@storageforcustomers.dfs.core.windows.net/customers.csv,customers.csv,4603,1730905207000
abfss://raw@storageforcustomers.dfs.core.windows.net/loan_payments.csv,loan_payments.csv,2613,1730905205000
abfss://raw@storageforcustomers.dfs.core.windows.net/loans.csv,loans.csv,2340,1730905209000
abfss://raw@storageforcustomers.dfs.core.windows.net/transactions.csv,transactions.csv,3513,1730905207000


In [0]:

df = spark.read \
    .option("header", "true") \
    .option("delimiter", "\t") \
    .csv("abfss://raw@storageforcustomers.dfs.core.windows.net/accounts.csv")
display(df)



"account_id,customer_id,account_type,balance"
"1,45,Savings,1000.50"
"2,12,Checking,2500.75"
"3,78,Savings,1500.00"
"4,34,Checking,3000.25"
"5,56,Savings,500.00"
"6,23,Checking,1200.50"
"7,89,Savings,800.75"
"8,67,Checking,2200.00"
"9,14,Savings,900.25"
"10,92,Checking,1800.50"


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, StringType, DoubleType
df_split = df.select(split(col("account_id,customer_id,account_type,balance"), ",").alias("split_cols")
)

df_modified = df_split.select(
    col("split_cols").getItem(0).cast(IntegerType()).alias("account_id"),
    col("split_cols").getItem(1).cast(IntegerType()).alias("customer_id"),
    col("split_cols").getItem(2).cast(StringType()).alias("account_type"),
    col("split_cols").getItem(3).cast(DoubleType()).alias("balance")
)
df_modified.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- account_type: string (nullable = true)
 |-- balance: double (nullable = true)



In [0]:
df_modified.show(20)

+----------+-----------+------------+-------+
|account_id|customer_id|account_type|balance|
+----------+-----------+------------+-------+
|         1|         45|     Savings| 1000.5|
|         2|         12|    Checking|2500.75|
|         3|         78|     Savings| 1500.0|
|         4|         34|    Checking|3000.25|
|         5|         56|     Savings|  500.0|
|         6|         23|    Checking| 1200.5|
|         7|         89|     Savings| 800.75|
|         8|         67|    Checking| 2200.0|
|         9|         14|     Savings| 900.25|
|        10|         92|    Checking| 1800.5|
|        11|          3|     Savings|1100.75|
|        12|         81|    Checking| 2700.0|
|        13|         29|     Savings|1300.25|
|        14|         64|    Checking| 3200.5|
|        15|         47|     Savings| 700.75|
|        16|         18|    Checking| 1400.0|
|        17|         99|     Savings| 600.25|
|        18|          5|    Checking| 1600.5|
|        19|         76|     Savin

In [0]:
df_modified.write.format("delta").mode("overwrite").save("abfss://raw@storageforcustomers.dfs.core.windows.net/cleaned_data/accounts.csv")


In [0]:
df_modified123 = spark.read.format("delta").load("abfss://raw@storageforcustomers.dfs.core.windows.net/cleaned_data/accounts.csv")
df_modified123.show(20)

+----------+-----------+------------+-------+
|account_id|customer_id|account_type|balance|
+----------+-----------+------------+-------+
|         1|         45|     Savings| 1000.5|
|         2|         12|    Checking|2500.75|
|         3|         78|     Savings| 1500.0|
|         4|         34|    Checking|3000.25|
|         5|         56|     Savings|  500.0|
|         6|         23|    Checking| 1200.5|
|         7|         89|     Savings| 800.75|
|         8|         67|    Checking| 2200.0|
|         9|         14|     Savings| 900.25|
|        10|         92|    Checking| 1800.5|
|        11|          3|     Savings|1100.75|
|        12|         81|    Checking| 2700.0|
|        13|         29|     Savings|1300.25|
|        14|         64|    Checking| 3200.5|
|        15|         47|     Savings| 700.75|
|        16|         18|    Checking| 1400.0|
|        17|         99|     Savings| 600.25|
|        18|          5|    Checking| 1600.5|
|        19|         76|     Savin

In [0]:
df_customers = spark.read \
    .option("header", "true") \
    .option("delimiter", "\t") \
    .csv("abfss://raw@storageforcustomers.dfs.core.windows.net/customers.csv")
display(df)

"account_id,customer_id,account_type,balance"
"1,45,Savings,1000.50"
"2,12,Checking,2500.75"
"3,78,Savings,1500.00"
"4,34,Checking,3000.25"
"5,56,Savings,500.00"
"6,23,Checking,1200.50"
"7,89,Savings,800.75"
"8,67,Checking,2200.00"
"9,14,Savings,900.25"
"10,92,Checking,1800.50"


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


df_split = df_customers.select(split(col("customer_id,first_name,last_name,address,city,state,zip"), ",").alias("split_cols"))

df_modified_1 = df_split.select(
    col("split_cols").getItem(0).cast(IntegerType()).alias("customer_id"),
    col("split_cols").getItem(1).cast(StringType()).alias("first_name"),
    col("split_cols").getItem(2).cast(StringType()).alias("last_name"),
    col("split_cols").getItem(3).cast(StringType()).alias("address"),
    col("split_cols").getItem(4).cast(StringType()).alias("city"),
    col("split_cols").getItem(5).cast(StringType()).alias("state"),
    col("split_cols").getItem(6).cast(StringType()).alias("zip")
)

df_modified_1.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)



In [0]:
df_modified_1.show(50)

+-----------+-----------+----------+----------------+--------------+-----+------+
|customer_id| first_name| last_name|         address|          city|state|   zip|
+-----------+-----------+----------+----------------+--------------+-----+------+
|          1|       John|       Doe|      123 Elm St|       Toronto|   ON|M4B1B3|
|          2|       Jane|     Smith|   456 Maple Ave|        Ottawa|   ON|K1A0B1|
|          3|    Michael|   Johnson|      789 Oak Dr|      Montreal|   QC|H1A1A1|
|          4|      Emily|     Davis|     101 Pine Rd|       Calgary|   AB|T2A0A1|
|          5|      David|    Wilson|  202 Birch Blvd|     Vancouver|   BC|V5K0A1|
|          6|       Emma|     Clark|    505 Cedar St|       Halifax|   NS|B3H0A1|
|          7|      James|  Martinez|   606 Spruce Ln|      Winnipeg|   MB|R3C0A1|
|          8|     Olivia|    Garcia|      707 Fir St|      Edmonton|   AB|T5A0A1|
|          9|    William|     Lopez|  808 Redwood Dr|      Victoria|   BC|V8W0A1|
|         10|   

In [0]:
df_modified_1.write.format("delta").mode("overwrite").save("abfss://raw@storageforcustomers.dfs.core.windows.net/cleaned_data/customers.csv")

In [0]:

df_lp = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .csv("abfss://raw@storageforcustomers.dfs.core.windows.net/loan_payments.csv")
display(df_lp)

payment_id,loan_id,payment_date,payment_amount
1,45,2024-01-01,100.00
2,23,2024-01-02,150.00
3,67,2024-01-03,200.00
4,89,2024-01-04,250.00
5,12,2024-01-05,300.00
6,34,2024-01-06,350.00
7,56,2024-01-07,400.00
8,78,2024-01-08,450.00
9,90,2024-01-09,500.00
10,11,2024-01-10,550.00


In [0]:

from pyspark.sql.types import DateType
df_lp = df_lp \
    .withColumn("payment_id", col("payment_id").cast(IntegerType())) \
    .withColumn("loan_id", col("loan_id").cast(IntegerType())) \
    .withColumn("payment_date", col("payment_date").cast(DateType())) \
    .withColumn("payment_amount", col("payment_amount").cast(DoubleType()))

df_lp.show()


+----------+-------+------------+--------------+
|payment_id|loan_id|payment_date|payment_amount|
+----------+-------+------------+--------------+
|         1|     45|  2024-01-01|         100.0|
|         2|     23|  2024-01-02|         150.0|
|         3|     67|  2024-01-03|         200.0|
|         4|     89|  2024-01-04|         250.0|
|         5|     12|  2024-01-05|         300.0|
|         6|     34|  2024-01-06|         350.0|
|         7|     56|  2024-01-07|         400.0|
|         8|     78|  2024-01-08|         450.0|
|         9|     90|  2024-01-09|         500.0|
|        10|     11|  2024-01-10|         550.0|
|        11|     22|  2024-01-11|         600.0|
|        12|     33|  2024-01-12|         650.0|
|        13|     44|  2024-01-13|         700.0|
|        14|     55|  2024-01-14|         750.0|
|        15|     66|  2024-01-15|         800.0|
|        16|     77|  2024-01-16|         850.0|
|        17|     88|  2024-01-17|         900.0|
|        18|     99|

In [0]:
df_lp.write.format("delta").mode("overwrite").save("abfss://raw@storageforcustomers.dfs.core.windows.net/cleaned_data/loan_payments.csv")


In [0]:
df_l = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .csv("abfss://raw@storageforcustomers.dfs.core.windows.net/loans.csv")
display(df_l)

loan_id,customer_id,loan_amount,interest_rate,loan_term
1,45,10000.50,5.5,36
2,12,20000.75,4.5,48
3,78,15000.00,6.0,60
4,34,30000.25,3.5,24
5,56,25000.00,5.0,36
6,23,17500.50,4.0,48
7,89,22500.75,6.5,60
8,67,27500.00,3.0,24
9,14,32500.25,5.5,36
10,92,37500.50,4.5,48


In [0]:
df_l = df_l \
    .withColumn("loan_id", col("loan_id").cast(IntegerType())) \
    .withColumn("customer_id", col("customer_id").cast(IntegerType())) \
    .withColumn("loan_amount", col("loan_amount").cast(DoubleType())) \
    .withColumn("loan_term", col("loan_term").cast(IntegerType())) \
    .withColumn("interest_rate", col("interest_rate").cast(DoubleType()))

df_l.show()


+-------+-----------+-----------+-------------+---------+
|loan_id|customer_id|loan_amount|interest_rate|loan_term|
+-------+-----------+-----------+-------------+---------+
|      1|         45|    10000.5|          5.5|       36|
|      2|         12|   20000.75|          4.5|       48|
|      3|         78|    15000.0|          6.0|       60|
|      4|         34|   30000.25|          3.5|       24|
|      5|         56|    25000.0|          5.0|       36|
|      6|         23|    17500.5|          4.0|       48|
|      7|         89|   22500.75|          6.5|       60|
|      8|         67|    27500.0|          3.0|       24|
|      9|         14|   32500.25|          5.5|       36|
|     10|         92|    37500.5|          4.5|       48|
|     11|          3|   10000.75|          6.0|       60|
|     12|         81|    20000.0|          3.5|       24|
|     13|         29|   15000.25|          5.0|       36|
|     14|         64|    30000.5|          4.0|       48|
|     15|     

In [0]:
df_lp.write.format("delta").mode("overwrite").save("abfss://raw@storageforcustomers.dfs.core.windows.net/cleaned_data/loans.csv")


In [0]:
df_t = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .csv("abfss://raw@storageforcustomers.dfs.core.windows.net/transactions.csv")
display(df_t)


transaction_id,account_id,transaction_date,transaction_amount,transaction_type
1,45,2024-01-01,100.50,Deposit
2,12,2024-01-02,200.75,Withdrawal
3,78,2024-01-03,150.00,Deposit
4,34,2024-01-04,300.25,Withdrawal
5,56,2024-01-05,250.00,Deposit
6,23,2024-01-06,175.00,Withdrawal
7,89,2024-01-07,225.50,Deposit
8,67,2024-01-08,275.75,Withdrawal
9,14,2024-01-09,325.00,Deposit
10,92,2024-01-10,375.25,Withdrawal


In [0]:


df_t = df_t \
    .withColumn("transaction_id", col("transaction_id").cast(IntegerType())) \
    .withColumn("account_id", col("account_id").cast(IntegerType())) \
    .withColumn("transaction_date", col("transaction_date").cast(DateType())) \
    .withColumn("transaction_amount", col("transaction_amount").cast(DoubleType())) \
    .withColumn("transaction_type", col("transaction_type").cast(StringType()))

df_t.show()


+--------------+----------+----------------+------------------+----------------+
|transaction_id|account_id|transaction_date|transaction_amount|transaction_type|
+--------------+----------+----------------+------------------+----------------+
|             1|        45|      2024-01-01|             100.5|         Deposit|
|             2|        12|      2024-01-02|            200.75|      Withdrawal|
|             3|        78|      2024-01-03|             150.0|         Deposit|
|             4|        34|      2024-01-04|            300.25|      Withdrawal|
|             5|        56|      2024-01-05|             250.0|         Deposit|
|             6|        23|      2024-01-06|             175.0|      Withdrawal|
|             7|        89|      2024-01-07|             225.5|         Deposit|
|             8|        67|      2024-01-08|            275.75|      Withdrawal|
|             9|        14|      2024-01-09|             325.0|         Deposit|
|            10|        92| 

In [0]:
df_t.write.format("delta").mode("overwrite").save("abfss://raw@storageforcustomers.dfs.core.windows.net/cleaned_data/transactions.csv")